In [1]:
import os
os.getcwd()

'C:\\Users\\Albert\\Desktop\\DATA SCIENCE\\UBIQUM\\UBIQUM_1X2_Python\\notebooks'

In [2]:
import pandas as pd

xl = pd.ExcelFile('../bbdd/FULL_BBDD_only_results.xlsx')
df = xl.parse(xl.sheet_names[0])

In [3]:
df.head()

,TEMPORADA,LIGA,JORNADA,FECHA,LOCAL,VISITANTE,GOL L,GOL V,ARBITRO,QUINIELA
0,1999,1,1,1999-08-21,Mallorca,Real Madrid,1,2,LOSANTOS OMAR,2
1,1999,1,1,1999-08-21,Athletic Club,Real Betis,1,0,MEJUTO GONZALEZ,1
2,1999,2,1,1999-08-21,Leganes,Mérida C.P.,1,1,NAVAS LASA,X
3,1999,1,1,1999-08-21,Valencia,Racing Santander,1,2,PRADOS GARCIA,2
4,1999,2,1,1999-08-21,U.E. Lleida,Villarreal,2,2,RUBIO INIESTA,X


In [4]:
maxdate = max(df.FECHA)
print(maxdate)

2019-06-09 00:00:00


### Import new data

Start by retrieving the whole current season

In [5]:
import requests
headers = {'x-rapidapi-key': "28d61524ab5698a1f8edb9e9ed577221"}

league_1 = requests.request("GET", "https://server1.api-football.com/fixtures/league/775", headers=headers).json()
league_2 = requests.request("GET", "https://server1.api-football.com/fixtures/league/776", headers=headers).json()

Get only games which are not in the database and are already finished

In [40]:
FECHA = []
LOCAL = []
VISITANTE = []
ARBITRO = []
GOL_L = []
GOL_V = []
TEMPORADA = []
LIGA = []
JORNADA = []  
QUINIELA = []

import datetime

for i in league_1['api']['fixtures']:
    if (datetime.datetime.strptime(i['event_date'][:10], '%Y-%m-%d') > maxdate) and (i['status']=='Match Finished'):
            FECHA.append(datetime.datetime.strptime(i['event_date'][:10], '%Y-%m-%d'))
            LOCAL.append(i['homeTeam']['team_name'])
            VISITANTE.append(i['awayTeam']['team_name'])
            ARBITRO.append('-') if i['referee'] == None else ARBITRO.append(i['referee'])
            GOL_L.append(i['goalsHomeTeam'])
            GOL_V.append(i['goalsAwayTeam'])
            JORNADA.append(int(i['round'][17:]))
            LIGA.append(1)
            TEMPORADA.append(2019)
            QUINIELA.append('1') if i['goalsHomeTeam'] > i['goalsAwayTeam'] else QUINIELA.append('2') \
                    if i['goalsHomeTeam'] < i['goalsAwayTeam'] else QUINIELA.append('X')
            
for i in league_2['api']['fixtures']:
    if (datetime.datetime.strptime(i['event_date'][:10], '%Y-%m-%d') > maxdate) and (i['status']=='Match Finished'):
            FECHA.append(datetime.datetime.strptime(i['event_date'][:10], '%Y-%m-%d'))
            LOCAL.append(i['homeTeam']['team_name'])
            VISITANTE.append(i['awayTeam']['team_name'])
            ARBITRO.append('-') if i['referee'] == None else ARBITRO.append(i['referee'])
            GOL_L.append(i['goalsHomeTeam'])
            GOL_V.append(int(i['goalsAwayTeam']))
            JORNADA.append(i['round'][17:])
            LIGA.append(2)
            TEMPORADA.append(2019)
            QUINIELA.append('1') if i['goalsHomeTeam'] > i['goalsAwayTeam'] else QUINIELA.append('2') \
                    if i['goalsHomeTeam'] < i['goalsAwayTeam'] else QUINIELA.append('X')

In [41]:
import pandas as pd

new = pd.DataFrame(list(zip(TEMPORADA, LIGA, JORNADA, FECHA, LOCAL, VISITANTE, GOL_L, GOL_V, ARBITRO, QUINIELA)), 
               columns =['TEMPORADA', 'LIGA', 'JORNADA', 'FECHA', 'LOCAL', 'VISITANTE', 'GOL L', 'GOL V', 
                         'ARBITRO', 'QUINIELA'])

### Check names with dictionary

Nothing should change with last dictionary during season, only with new season could appear new referees or teams

In [8]:
import json

with open('../bbdd/teams_dict.json') as json_file:
    teams_dict = json.load(json_file)
    
with open('../bbdd/referees_dict.json') as json_file:
    referees_dict = json.load(json_file)

Update dictionary with new teams and referees

In [9]:
save_r = False
for i in new['ARBITRO']:
    if i not in referees_dict:
        referees_dict.update({i:i})
        save_r = True

In [10]:
save_t = False
for i in new['LOCAL']:
    if i not in teams_dict:
        teams_dict.append({i:i})
        save_t = True

for i in new['VISITANTE']:
    if i not in teams_dict:
        teams_dict.append({i:i})
        save_t = True

In [11]:
new['LOCAL'] = [teams_dict[i] for i in new['LOCAL']]
new['VISITANTE'] = [teams_dict[i] for i in new['VISITANTE']]
new['ARBITRO'] = [referees_dict[i] for i in new['ARBITRO']]

In [12]:
print(save_r, save_t)

False False


In [13]:
if save_r:
    with open('../bbdd/referees_dict.json', 'w') as outfile:
            json.dump(referees_dict, outfile)
    
if save_t:
    with open('../bbdd/teams_dict.json', 'w') as outfile:
            json.dump(teams_dict, outfile)

### Update simpe dataframe with results

Creo un nuevo dataframe que combina bbdd con nuevos partidos

In [14]:
df_new = df.append(new, ignore_index=True, sort=False)

In [33]:
df_new.to_excel("../bbdd/FULL_BBDD_only_results.xlsx", index=False)

### Create additional features for new games

Load the functions to create the additional features

In [15]:
# last n games of home or away team

def last_games(df, league, team, date, season, ngames):
    select = df[(df['LIGA']==league) & ((df['LOCAL']==team) | (df['VISITANTE']==team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)][:ngames]
    games = select.shape[0] + 1/2
    WINS = (sum(select['QUINIELA'][select.LOCAL==team]=='1') + sum(select['QUINIELA'][select.VISITANTE==team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    LOSE = (1 - WINS - DRAW)
    return(WINS, DRAW, LOSE)

In [16]:
# total league. local as a local

def league_local_local(df, league, home_team, date, season):
    select = df[(df['LIGA']==league) & ((df['LOCAL']==home_team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)]
    games = select.shape[0] + 1/2
    H_WINS = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 1/6)/games
    H_DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    H_LOSE = (1 - H_WINS - H_DRAW)
    return(H_WINS, H_DRAW, H_LOSE)

In [17]:
# total league. visitor as a visitor

def league_visitor_visitor(df, league, away_team, date, season):
    select = df[(df['LIGA']==league) & ((df['VISITANTE']==away_team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)]
    games = select.shape[0] + 1/2
    V_WINS = (sum(select['QUINIELA'][select.VISITANTE==away_team]=='2') + 1/6)/games
    V_DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    V_LOSE = (1 - V_WINS - V_DRAW)
    return(V_WINS, V_DRAW, V_LOSE)

In [18]:
# h2h last 10 games total

def h2h_10_hda(df, date, home_team, away_team):
    select = df[(df['FECHA'] < date) & (((df['LOCAL']==home_team) & (df['VISITANTE']==away_team)) |  
            ((df['LOCAL']==away_team) & (df['VISITANTE']==home_team)))][:10]
    games = select.shape[0] + 1/2
    HOME = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 
            sum(select['QUINIELA'][select.VISITANTE==home_team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    AWAY = (1 - HOME - DRAW)
    return(HOME, DRAW, AWAY)

In [19]:
# h2h last 5 games with same home and away

def h2h_5_homeaway(df, date, home_team, away_team):
    select = df[(df['FECHA'] < date) & (df['LOCAL']==home_team) & (df['VISITANTE']==away_team)][:5]
    games = select.shape[0] + 1/2
    HOME = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 1/6)/games
    DRAW = (sum(select['QUINIELA'][select.LOCAL==home_team]=='X') + 1/6)/games
    AWAY = (1 - HOME - DRAW)
    return(HOME, DRAW, AWAY)

In [20]:
# referee: historical results

def ref_team(df, date, team, referee):
    select = df[(df['FECHA'] < date) & ((df['LOCAL']==team) | (df['VISITANTE']==team)) & (df['ARBITRO']==referee)]
    games = select.shape[0] + 1/2
    WINS = (sum(select['QUINIELA'][select.LOCAL==team]=='1') + 
            sum(select['QUINIELA'][select.VISITANTE==team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    LOSE = (1 - WINS - DRAW)
    return(WINS, DRAW, LOSE)

In [21]:
# referee: historical results with home team at home

def ref_home_home(df, date, home_team, referee):
    select = df[(df['FECHA'] < date) & (df['LOCAL']==home_team) & (df['ARBITRO']==referee)]
    games = select.shape[0] + 1/2
    WINS = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 
            sum(select['QUINIELA'][select.VISITANTE==home_team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    LOSE = (1 - WINS - DRAW)
    return(WINS, DRAW, LOSE)

In [22]:
# referee: historical results with away team playing away

def ref_away_away(df, date, away_team, referee): 
    select = df[(df['FECHA'] < date) & (df['VISITANTE']==away_team) & (df['ARBITRO']==referee)]
    games = select.shape[0] + 1/2
    WINS = (sum(select['QUINIELA'][select.LOCAL==away_team]=='1') + 
            sum(select['QUINIELA'][select.VISITANTE==away_team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    LOSE = (1 - WINS - DRAW)
    return(WINS, DRAW, LOSE)

In [23]:
new['last2_hometeam_wdl'] = new.apply(lambda x: last_games(df_new, x[1], x[4], x[3], x[0], 2), axis=1)
new['last5_hometeam_wdl'] = new.apply(lambda x: last_games(df_new, x[1], x[4], x[3], x[0], 5), axis=1)
new['last10_hometeam_wdl'] = new.apply(lambda x: last_games(df_new, x[1], x[4], x[3], x[0], 10), axis=1)

In [24]:
new['last2_awayteam_wdl'] = new.apply(lambda x: last_games(df_new, x[1], x[5], x[3], x[0], 2), axis=1)
new['last5_awayteam_wdl'] = new.apply(lambda x: last_games(df_new, x[1], x[5], x[3], x[0], 5), axis=1)
new['last10_awayteam_wdl'] = new.apply(lambda x: last_games(df_new, x[1], x[5], x[3], x[0], 10), axis=1)

In [25]:
new['league_local_local_wdl'] = new.apply(lambda x: league_local_local(df_new, x[1], x[4], x[3], x[0]), axis=1)
new['league_visitor_visitor_wdl'] = new.apply(lambda x: league_visitor_visitor(df_new, x[1], x[5], x[3], x[0]), axis=1)

In [26]:
new['h2h_last_10_hda'] = new.apply(lambda x: h2h_10_hda(df_new, x[3], x[4], x[5]), axis=1)
new['h2h_5_homeaway_hda'] = new.apply(lambda x: h2h_5_homeaway(df_new, x[3], x[4], x[5]), axis=1)

In [27]:
new['ref_hometeam_wdl'] = new.apply(lambda x: ref_team(df_new, x[3], x[4], x[8]), axis=1)
new['ref_awayteam_wdl'] = new.apply(lambda x: ref_team(df_new, x[3], x[5], x[8]), axis=1)

In [28]:
new['ref_home_home_wdl'] = new.apply(lambda x: ref_home_home(df_new, x[3], x[4], x[8]), axis=1)
new['ref_away_away_wdl'] = new.apply(lambda x: ref_away_away(df_new, x[3], x[5], x[8]), axis=1)

### Split tuples into columns and delete tuples

In [29]:
columns = list(new.columns[[x[-3:] in ['wdl','hda'] for x in new.columns]])

In [30]:
for i in columns:
    field_type = i[-3:]
    name1 = i[:-3] + field_type[0]
    name2 = i[:-3] + field_type[1]
    name3 = i[:-3] + field_type[2]
    
    new[name1] = new[i].apply(lambda x: x[0])
    new[name2] = new[i].apply(lambda x: x[1])
    new[name3] = new[i].apply(lambda x: x[2])

In [31]:
new.drop(columns, inplace=True, axis=1)

In [32]:
new.columns

Index(['TEMPORADA', 'LIGA', 'JORNADA', 'FECHA', 'LOCAL', 'VISITANTE', 'GOL L',
       'GOL V', 'ARBITRO', 'QUINIELA', 'last2_hometeam_w', 'last2_hometeam_d',
       'last2_hometeam_l', 'last5_hometeam_w', 'last5_hometeam_d',
       'last5_hometeam_l', 'last10_hometeam_w', 'last10_hometeam_d',
       'last10_hometeam_l', 'last2_awayteam_w', 'last2_awayteam_d',
       'last2_awayteam_l', 'last5_awayteam_w', 'last5_awayteam_d',
       'last5_awayteam_l', 'last10_awayteam_w', 'last10_awayteam_d',
       'last10_awayteam_l', 'league_local_local_w', 'league_local_local_d',
       'league_local_local_l', 'league_visitor_visitor_w',
       'league_visitor_visitor_d', 'league_visitor_visitor_l', 'h2h_last_10_h',
       'h2h_last_10_d', 'h2h_last_10_a', 'h2h_5_homeaway_h',
       'h2h_5_homeaway_d', 'h2h_5_homeaway_a', 'ref_hometeam_w',
       'ref_hometeam_d', 'ref_hometeam_l', 'ref_awayteam_w', 'ref_awayteam_d',
       'ref_awayteam_l', 'ref_home_home_w', 'ref_home_home_d',
       'ref_home_ho

In [35]:
new.shape

(388, 52)

In [36]:
xl = pd.ExcelFile('../bbdd/FULL_BBDD_with_features.xlsx')
df = xl.parse(xl.sheet_names[0])

In [37]:
df_new = df.append(new, ignore_index=True, sort=False)

In [38]:
df_new.shape

(17228, 52)

In [39]:
df_new.to_excel("../bbdd/FULL_BBDD_with_features.xlsx", index=False)